In [1]:
import h2o
from h2o.automl import H2OAutoML

import pandas as pd
import numpy as np

import matplotlib as plt
%matplotlib inline

#Import the Estimators
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators import H2OXGBoostEstimator

#Import h2o grid search 
import h2o.grid 
from h2o.grid.grid_search import H2OGridSearch

In [2]:
h2o.init(max_mem_size = "40g", nthreads = 11)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.4" 2024-07-16; OpenJDK Runtime Environment (build 21.0.4+7-Ubuntu-1ubuntu224.04); OpenJDK 64-Bit Server VM (build 21.0.4+7-Ubuntu-1ubuntu224.04, mixed mode, sharing)
  Starting server from /home/mark/for_pycaret/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfgsgoswu
  JVM stdout: /tmp/tmpfgsgoswu/h2o_mark_started_from_python.out
  JVM stderr: /tmp/tmpfgsgoswu/h2o_mark_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,3 months and 14 days
H2O_cluster_name:,H2O_from_python_mark_keph72
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,39.98 Gb
H2O_cluster_total_cores:,128
H2O_cluster_allowed_cores:,11
H2O_cluster_status:,"locked, healthy"


In [6]:
data = h2o.import_file("data/STAGE_1_TRAIN.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
x = data.columns
y = "tow"
ignore = ["flight_id", "dataset"] 
## add adep and ades?
x = list(set(x) - set(ignore))
x.remove(y)
print(x)

['mtow_fill', 'first_cruise_alt', 'adep', 'oew_fill', 'great_circle_km', 'ades', 'month', 'airline', 'day_of_week', 'flight_duration_sec', 'replacer', 'hour_in_local', 'est_tow', 'total_fuel_fill', 'est_load_lf_adjusted', 'aircraft_type']


In [8]:
data["airline"] = data["airline"].asfactor()
data["aircraft_type"] = data["aircraft_type"].asfactor()
data["replacer"] = data["replacer"].asfactor()
data["adep"] = data["adep"].asfactor()
data["ades"] = data["ades"].asfactor()
data["month"] = data["month"].asfactor()
data["day_of_week"] = data["day_of_week"].asfactor()
data["hour_in_local"] = data["hour_in_local"].asfactor()

In [9]:
test, train = data.split_frame(ratios = [0.05], seed = 26)
print(len(train))

350583


In [24]:
train.as_data_frame().to_csv("data/train_actual.csv", index=False)

/home/mark/for_pycaret/lib/python3.11/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [10]:
aml = H2OAutoML(max_models=40, seed=75, verbosity="info", stopping_metric="RMSE")
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
02:29:07.638: Project: AutoML_1_20241024_22907
02:29:07.658: 5-fold cross-validation will be used.
02:29:07.659: Setting stopping tolerance adaptively based on the training frame: 0.0016889024802299066
02:29:07.660: Build control seed: 75
02:29:07.664: training frame: Frame key: AutoML_1_20241024_22907_training_py_11_sid_a0b8    cols: 21    rows: 350583  chunks: 512    size: 19087068  checksum: 1909494427907525494
02:29:07.664: validation frame: NULL
02:29:07.665: leaderboard frame: NULL
02:29:07.666: blending frame: NULL
02:29:07.669: response column: tow
02:29:07.670: fold column: null
02:29:07.671: weights column: null
02:29:08.12: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4g, 60w), lr_annealing (7g, 10w)]}, {D

key,value
Stacking strategy,cross_validation
Number of base models (used / total),17/40
# GBM base models (used / total),8/12
# XGBoost base models (used / total),8/14
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),0/11
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [20]:
leader_model = aml.leader
print(round(leader_model.rmse(), 0))
#print(round(leader_model.rmse(valid=True), 0))
print(round(leader_model.model_performance(test).rmse(), 0))

2406.0
3146.0


In [21]:
# 2036.0
# 2753.0

In [22]:
## Save ensemble
from datetime import datetime
now = datetime.now()
h2o.save_model(model=leader_model, path="data/models/stage_one_"+now.strftime("%m_%d_%y_%H_%M")+"_added", force=True)

'/home/mark/prc_challenge/data/models/stage_one_10_24_24_17_16_added/StackedEnsemble_AllModels_1_AutoML_1_20241024_22907'

In [13]:
# saved_model = h2o.load_model("/home/mark/prc_challenge/data/models/stage_one_10_17_24_22_18_added/StackedEnsemble_AllModels_1_AutoML_1_20241017_151109")
# saved_model

In [19]:
aml.get_leaderboard().as_data_frame().to_csv("data/saved_leaderboard_StackedEnsemble_AllModels_1_AutoML_1_20241024_22907.csv", index=False)

/home/mark/for_pycaret/lib/python3.11/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [18]:
for m in aml.leaderboard.as_data_frame()['model_id']:
    print(m)
    model = h2o.get_model(m)
    try: 
        importance = model.varimp(use_pandas=True)
        print(importance) 
    except: 
        print("no var data")


/home/mark/for_pycaret/lib/python3.11/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


StackedEnsemble_AllModels_1_AutoML_1_20241024_22907
None
StackedEnsemble_BestOfFamily_1_AutoML_1_20241024_22907
None
GBM_1_AutoML_1_20241024_22907
                variable  relative_importance  scaled_importance  percentage
0                est_tow         2.220152e+15           1.000000    0.437850
1              mtow_fill         1.409942e+15           0.635065    0.278063
2               oew_fill         6.349268e+14           0.285984    0.125218
3          aircraft_type         6.221515e+14           0.280229    0.122698
4        great_circle_km         4.498032e+13           0.020260    0.008871
5                airline         3.224160e+13           0.014522    0.006359
6                   adep         2.687007e+13           0.012103    0.005299
7        total_fuel_fill         2.361005e+13           0.010634    0.004656
8                   ades         1.528534e+13           0.006885    0.003015
9               replacer         1.435420e+13           0.006465    0.002831
10  es